In [141]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [142]:
dev = qml.device('default.qubit', wires=5)

In [143]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.RY(np.pi / 6, wires=1)
    qml.CNOT(wires=[0, 1])
    qml.Rot(*angles, wires=0)

In [144]:
def generator(w, **kwargs):
    qml.Hadamard(wires=2)
    qml.RX(w[0], wires=2)
    qml.RX(w[1], wires=3)
    qml.RY(w[2], wires=2)
    qml.RY(w[3], wires=3)
    qml.RZ(w[4], wires=2)
    qml.RZ(w[5], wires=3)
    qml.CNOT(wires=[2, 3])
    qml.RX(w[6], wires=2)
    qml.RY(w[7], wires=2)
    qml.RZ(w[8], wires=2)

In [145]:
def swap_test():
    qml.Hadamard(wires=4)
    qml.CSWAP(wires=[4, 0, 2])
    qml.Hadamard(wires=4)

In [146]:
@qml.qnode(dev, interface="tf")
def overlap(phi, theta, omega, gen_weights):
    real([phi, theta, omega])
    generator(gen_weights)
    swap_test()
    return qml.expval(qml.PauliZ(4))

In [147]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))

gen_weights = tf.Variable(init_gen_weights)

In [148]:
opt = tf.keras.optimizers.SGD(0.4)

In [149]:
cost = lambda: 1 - overlap(phi, theta, omega, gen_weights)
for step in range(50):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 0.47903778021552934
Step 5: cost = 0.38517746997223345
Step 10: cost = 0.3161302288548482
Step 15: cost = 0.27869552611104986
Step 20: cost = 0.26216292759087967
Step 25: cost = 0.25529962583341426
Step 30: cost = 0.2523962691481132
Step 35: cost = 0.25111593770595764
Step 40: cost = 0.2505299313354066
Step 45: cost = 0.25025464969337974


In [150]:
obs_real = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]
obs_gen = [qml.PauliX(2), qml.PauliY(2), qml.PauliZ(2)]

bloch_vector_real = qml.map(real, obs_real, dev, interface="tf")
bloch_vector_generator = qml.map(generator, obs_gen, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real([phi, theta, omega])))
print("Generator Bloch vector: {}".format(bloch_vector_generator(gen_weights)))

Real Bloch vector: [-0.10847093  0.22524222 -0.4330127 ]
Generator Bloch vector: [-0.19959091  0.44361004 -0.87328997]
